## 1. Test LLM and prompt types

In [1]:
from configuration.llm_inference_configuration import APILLMConfiguration
from llm.azure_openai import AzureOpenAILLM
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = AzureOpenAILLM(gpt_4o_azure_configuration)

FileNotFoundError: [Errno 2] No such file or directory: 'configuration/yaml/llm/gpt_4o.yaml'

In [ ]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Health check. Say 'hi' to start the conversation."
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

In [ ]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

## 2. Test tools

### 2.1. Python code runner tool

In [ ]:
from tools.python_code_runner import PythonCodeRunnerTool
from configuration.tool_configuration import ToolConfiguration

In [ ]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [ ]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = python_code_runner.execute(input_code_list=input_code_list)
results

## 3. Test memory

In [ ]:
from configuration.llm_inference_configuration import APILLMConfiguration
from llm.azure_openai import AzureOpenAILLM
from base_classes.memory.memory_atom import AbstractMemoryAtom
from base_classes.memory.memory_block import AbstractMemoryBlock
from base_classes.memory.memory import AbstractMemory
from base_classes.memory.datatypes.data_item import PromptDataItem
from prompt.user_message import UserMessagePrompt
from prompt.assistant_message import AssistantMessagePrompt
from base_classes.memory.memory_feature_engineer import MemoryFeatureEngineer

In [2]:
prompt_data_1 = [{
    'role': 'user',
    'content': 'Calculate the sum of 11 and 20.'
}]
mem_atom_1 = AbstractMemoryAtom(data=PromptDataItem(UserMessagePrompt(prompt_data_1)))

prompt_data_2 = [{
    'role': 'assistant',
    'content': 'In order to calculate the sum of two numbers, we need to do the following:\n\t1. Write a Python code with add_sum() function, receiving 2 variables.\n\t2. Execute the code with the given variables.\n\t3. Return the result of the sum.'
}]
mem_atom_2 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_2)))

prompt_data_3 = [{
    'role': 'assistant',
    'content': 'Here is the Python code to calculate the sum of two numbers:\n\na = 11\nb = 20\nresult = a + b\n\nprint(f"The sum of {a} and {b} is {result}.")'
}]
mem_atom_3 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_3)))

prompt_data_4 = [{
    'role': 'assistant',
    'content': 'After executing the code, we got the sum of 11 and 20 is 31.'
}]
mem_atom_4 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_4)))

In [3]:
AbstractMemoryAtom.get_mematom_ids()

[UUID('502d1013-044e-46c0-b3e1-755f5e4cfd1c'),
 UUID('a13de8b8-ec81-4af4-a6bf-33f6e7c93b33'),
 UUID('ed6b3e6e-bcd4-4a97-8566-7fd562983cd3'),
 UUID('18fd9901-2553-43fe-bbf6-f9ce14418a89')]

In [4]:
mem_block = AbstractMemoryBlock()
mem_block.add_memory_atom(mem_atom_1)
mem_block.add_memory_atom(mem_atom_2)
mem_block.add_memory_atom(mem_atom_3)
mem_block.add_memory_atom(mem_atom_4)

mem_block.mem_atom_graph = {
        mem_atom_1.mem_atom_id: [mem_atom_2.mem_atom_id],
        mem_atom_2.mem_atom_id: [mem_atom_3.mem_atom_id, mem_atom_4.mem_atom_id],
        mem_atom_3.mem_atom_id: [],
        mem_atom_4.mem_atom_id: []
    }

In [6]:
memory = AbstractMemory()
memory.add_memory_block(mem_block)

AttributeError: 'AbstractMemory' object has no attribute '_memory_fe'

In [ ]:
memory.get_memory_block_by_id(mem_block.mem_block_id)

In [17]:
from annotated_docs.json_schema import as_json_schema

class A:
    """
    This is a class A.
    """
    
    def __init__(self, a: int, b: str):
        """
        This is the constructor of class A.
        """
        self.a = a
        self.b = b
    @classmethod
    def get_a(self, m: int) -> int:
        """
        This is the get_a method of class A.
        """
        return self.a
    
    def get_b(self) -> str:
        """
        This is the get_b method of class A.
        """
        return self.b

a = A(a = 2, b = "a")

as_json_schema(A.get_a)
# a = A(a = 2, b = "a")
# as_json_schema(A.get_a())

{'name': 'get_a',
 'description': 'This is the get_a method of class A.',
 'parameters': {'properties': {'m': {'type': 'integer'}},
  'required': ['m'],
  'type': 'object'}}